In [ ]:
data_path = '/Users/ryoungblood/image_similarity/original_images/'

In [ ]:
data_path = '/Users/ryoungblood/image_similarity/kris_images/Winchester/'

In [ ]:
%matplotlib inline

import os
import cv2
import random
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy
import math

In [ ]:
img_bgr = cv2.imread(os.path.join(data_path,'united'))
img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

## Base Image

In [ ]:
plt.imshow(img_rgb)

## Image Variations

### Single Channel (Luminosity) - Grayscale

In [ ]:
img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)

In [ ]:
plt.imshow(img_gray, cmap='gray')

### Single Channel (Luminosity) - Jet

In [ ]:
plt.imshow(img_gray, cmap='jet')

### Resize Each Axis by X%

In [ ]:
img_rgb_resizeXpct = cv2.resize(img_rgb, dsize=(0,0), fx=5, fy=5)
plt.imshow(img_rgb_resizeXpct)

In [ ]:
img_bgr_resizeXpct = cv2.cvtColor(img_rgb_resizeXpct,cv2.COLOR_RGB2BGR)

In [ ]:
cv2.imwrite(os.path.join(data_path,'united_5.jpg'),img_bgr_resizeXpct)

In [ ]:
img_rgb_rs_100pct = cv2.resize(img_rgb_resizeXpct, dsize=(0,0),
                               fx=2.0, fy=2.0)
plt.imshow(img_rgb_rs_100pct)

### Resize to Arbitrary 500x500

In [ ]:
img_rgb_thumb_500_500 = cv2.resize(img_rgb, dsize=(500,500))
plt.imshow(img_rgb_thumb_500_500)

### Rotate Random Angle

In [ ]:
# rotate function from http://john.freml.in/opencv-rotation
def rotate_about_center(src, angle, scale=1.):
    w = src.shape[1]
    print('w: ',w)
    h = src.shape[0]
    print('h: ',h)
    rangle = np.deg2rad(angle)  # angle in radians
    # now calculate new image width and height
    nw = (abs(np.sin(rangle)*h) + abs(np.cos(rangle)*w))*scale
    print('nw: ',nw)
    nh = (abs(np.cos(rangle)*h) + abs(np.sin(rangle)*w))*scale
    print('nh: ',nh)
    # ask OpenCV for the rotation matrix
    rot_mat = cv2.getRotationMatrix2D((nw*0.5, nh*0.5), angle, scale)
    # calculate the move from the old center to the new center combined
    # with the rotation
    rot_move = np.dot(rot_mat, np.array([(nw-w)*0.5, (nh-h)*0.5,0]))
    # the move only affects the translation, so update the translation
    # part of the transform
    rot_mat[0,2] += rot_move[0]
    rot_mat[1,2] += rot_move[1]
    return cv2.warpAffine(src, rot_mat, (int(math.ceil(nw)), int(math.ceil(nh))), flags=cv2.INTER_LANCZOS4)

In [ ]:
#rows, cols = img_rgb.shape[:2]
angle = random.randrange(0,360,90)
print(angle)
#M = cv2.getRotationMatrix2D((cols/2,cols/2),angle,1)
#img_rgb_rot = cv2.warpAffine(img_rgb,M,(rows,cols))
img_rgb_rot = rotate_about_center(img_rgb,angle)
plt.imshow(img_rgb_rot)
cv2.imwrite('/Users/ryoungblood/image_similarity/original_images/rotated.jpg',img_rgb_rot)

### Center Image Subset

In [ ]:
x = .8 # will take the center x percent of each axis to form subset
img_rgb_sub = img_rgb[
    int(round((1-x)/2*img_rgb.shape[0],0)):int(round((x+(1-x)/2)*img_rgb.shape[0],0)),
    int(round((1-x)/2*img_rgb.shape[1],0)):int(round((x+(1-x)/2)*img_rgb.shape[1],0))
]
plt.imshow(img_rgb_sub)

### Gaussian Blur

In [ ]:
img_rgb_gauss = cv2.GaussianBlur(img_rgb, (13,13), 0)
plt.imshow(img_rgb_gauss)

### Noise Added

In [ ]:
img = cv2.imread('/Users/ryoungblood/opencv_tests/data/gradient.jpg',0)
print(img.shape)
print(img[:10,:10])

ret,thresh1 = cv2.threshold(img,127,255,cv2.THRESH_BINARY)
ret,thresh2 = cv2.threshold(img,127,255,cv2.THRESH_BINARY_INV)
ret,thresh3 = cv2.threshold(img,127,255,cv2.THRESH_TRUNC)
ret,thresh4 = cv2.threshold(img,127,255,cv2.THRESH_TOZERO)
ret,thresh5 = cv2.threshold(img,127,255,cv2.THRESH_TOZERO_INV)

titles = ['Original Image','BINARY','BINARY_INV','TRUNC','TOZERO','TOZERO_INV']
images = [img, thresh1, thresh2, thresh3, thresh4, thresh5]

for i in range(6):
    plt.subplot(2,3,i+1),plt.imshow(images[i],'gray')
    plt.title(titles[i])
    plt.xticks([]),plt.yticks([])

plt.show()


### Random Image Subset

In [ ]:
img = cv2.imread(os.path.join(data_path,'01.jpg'))
rows, cols = img.shape[:2]
print(rows)
print(cols)

In [ ]:
# will take x_size_pct percent of the x-axis, starting at column x_loc
x_pct_range = (.1,.9) 
x_size_pct = random.uniform(x_pct_range[0],x_pct_range[1]) 
x_size = int(round(x_size_pct * cols))
print("x size %: ",x_size_pct)
print("x size:   ",x_size)

y_pct_range = (.1,.9)
y_size_pct = random.uniform(y_pct_range[0],y_pct_range[1])
y_size = int(round(y_size_pct * rows))
print("y size %: ",y_size_pct)
print("y size:   ",y_size)

#if x_size is 0.7, then x_loc (the left-bound of the window) can be 0-0.3
x_loc_pct = random.uniform(0,1-x_size_pct)
x_loc = int(round(x_loc_pct * cols))
print("x loc %: ",x_loc_pct)
print("x loc:   ",x_loc)
y_loc_pct = random.uniform(0,1-y_size_pct)
y_loc = int(round(y_loc_pct * rows))
print("y loc %: ",y_loc_pct)
print("y loc:   ",y_loc)

img_overlay = deepcopy(img)
img_overlay[y_loc:(y_loc+y_size),x_loc] = (0,0,255)
img_overlay[y_loc:(y_loc+y_size),x_loc+x_size] = (0,0,255)
img_overlay[y_loc,x_loc:(x_loc+x_size)] = (0,0,255)
img_overlay[y_loc+y_size,x_loc:(x_loc+x_size)] = (0,0,255)
plt.imshow(cv2.cvtColor(img_overlay,cv2.COLOR_BGR2RGB))
plt.show()
img_rsub = img[y_loc:(y_loc+y_size),x_loc:(x_loc+x_size)]
plt.imshow(cv2.cvtColor(img_rsub,cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
img_0 = rotate_about_center(img_bgr,0)
img_90 = rotate_about_center(img_bgr,90)
img_180 = rotate_about_center(img_bgr,180)
img_270 = rotate_about_center(img_bgr,270)
cv2.imwrite('/Users/ryoungblood/image_similarity/original_images/rotated0.jpg',img_0)
cv2.imwrite('/Users/ryoungblood/image_similarity/original_images/rotated90.jpg',img_90)
cv2.imwrite('/Users/ryoungblood/image_similarity/original_images/rotated180.jpg',img_180)
cv2.imwrite('/Users/ryoungblood/image_similarity/original_images/rotated270.jpg',img_270)

In [ ]:
print(img_0.shape)
print(img_90.shape)
print(img_180.shape)
print(img_270.shape)